In [ ]:
import cv2
import glob
import numpy as np
import scipy.io as sio
from scipy.stats import skew
from scipy.stats import kurtosis
import pywt
from skimage.feature import greycomatrix
import scipy.io as sio
import numpy as np
import seaborn as sn
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from sklearn.decomposition import KernelPCA
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import itertools
import os
from sklearn.metrics import roc_curve, auc
kl = keras.layers
import skimage
import random
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)
# tf.keras.utils.set_random_seed(seed_value)
# tf.random.set_seed()

In [ ]:
from utils import*
# =============================================================================
# source_dir
# =============================================================================
# source_dir='C://Users//Kim-Ngoc//Desktop//CODE//'
source_dir= 'D://Fundus//Lung sound//V5_mspec_8s_4gr//'
# =============================================================================
# load mat files
# =============================================================================
covid_features=sio.loadmat(os.path.join(source_dir,'2gr_abnormal_322.mat')) 
covid_features=covid_features['abnormal'] 

normal_features=sio.loadmat(os.path.join(source_dir,'2gr_normal_322.mat')) 
normal_features=normal_features['normal']  

# pneumonia_features=sio.loadmat(os.path.join(source_dir,'pneumonia.mat')) 
# pneumonia_features=pneumonia_features['pneumonia']    
# =============================================================================
# devide feature pool into inputs and target labels
# =============================================================================
X=np.concatenate((covid_features[:,:-1],normal_features[:,:-1]), axis=0)#inputs
y=np.concatenate((covid_features[:,-1],normal_features[:,-1]), axis=0)#target labels
# =============================================================================
# normalization
# =============================================================================
min_max_scaler=MinMaxScaler()
X = min_max_scaler.fit_transform(X) 
# =============================================================================
# feature reduction (K-PCA)
# =============================================================================
transformer = KernelPCA(n_components=128, kernel='linear')
X = transformer.fit_transform(X)
y = to_categorical(y)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as kl
from sklearn.model_selection import train_test_split
tf.random.set_seed(42)
# Define function to build model with specified random seed
def build_model(feature_size, n_classes, seed_value, dropout):
    """ Build a small model for multi-label classification """
    inp = kl.Input((feature_size,))
    x = kl.Dense(512, activation='relu')(inp)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(512, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(512, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(256, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(128, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(64, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    x = kl.Dense(32, activation='relu')(x)
    x = kl.BatchNormalization()(x)
    x = kl.Dropout(dropout)(x)
    out = kl.Dense(n_classes, activation='softmax')(x)
    model = keras.Model(inputs=inp, outputs=out)
    model.summary()
    return model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy, Precision, Recall

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
precisions = []
recalls = []
confusion_matrices = []

best_test_acc = 0.0
best_model = None

for i, (train_index, val_index) in enumerate(kfold.split(X, y.argmax(1))):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    # y_train = y_train.ravel()
    # y_val = y_val.ravel()

    # Build and compile the model
    model = build_model(feature_size=X_train.shape[-1], n_classes=y_train.shape[-1], seed_value=42, dropout=0.1)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=[binary_accuracy, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val))
    val_metrics = model.evaluate(X_val, y_val)
    test_acc = val_metrics[1]
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_model = model
    # Evaluate the model on the validation set
    val_metrics = best_model.evaluate(X_val, y_val)
    accuracies.append(val_metrics[1])
    precisions.append(val_metrics[2])
    recalls.append(val_metrics[3])

    # Compute the confusion matrix
    y_pred = best_model.predict(X_val)
    y_pred = np.argmax(y_pred, axis=1)
    y_pred = (y_pred > 0.5).astype(int)
    cm = confusion_matrix(np.argmax(y_val, axis=1),y_pred)
    confusion_matrices.append(cm)

    # Save the model
    best_model.save(f'./model_seed/Linet/Main_V2/feature_128/5f_bs_128_v1/model_{i}.h5')
    # Reset the model for the next fold
    tf.keras.backend.clear_session()

In [ ]:
# Calculate the average metrics across all folds
avg_accuracy = sum(accuracies) / len(accuracies)
avg_precision = sum(precisions) / len(precisions)
avg_recall = sum(recalls) / len(recalls)

print('Average Accuracy:', avg_accuracy)
print('Average Precision:', avg_precision)
print('Average Recall:', avg_recall)

# Calculate the average confusion matrix across all folds
avg_cm = sum(confusion_matrices) / len(confusion_matrices)
print('Average Confusion Matrix:')
print(avg_cm)

In [ ]:
se = avg_cm[0][0]/(avg_cm[1][0] + avg_cm[0][0])
sp = avg_cm[1][1]/(avg_cm[0][1] + avg_cm[1][1])
sc = (se+sp)/2
print('Specificity Sp:', sp)
print('Sensitivity Se:', se)
print('Score Sc:', sc)

In [ ]:
# Save the results in a .txt file
with open('./model_seed/Linet/Main_V2/feature_128/5f_bs_128_v1/results_5folds.txt', 'w') as file:
    file.write('Test accuracy: {}\n'.format(avg_accuracy))
    file.write('Test precision: {}\n'.format(avg_precision))
    file.write('Test recall: {}\n'.format(avg_recall))
    file.write('Test F1 score: {}\n'.format(2*avg_precision*avg_recall/(avg_recall + avg_precision)))
    file.write('Specificity Sp: {}\n'.format(sp))
    file.write('Sensitivity Se: {}\n'.format(se))
    file.write('Score Sc: {}\n'.format(sc))
    file.write('CM: {}\n'.format(avg_cm))